# Подключение библиотек

In [1]:
from typing import Any

import numpy as np
from scipy.linalg import norm, block_diag, solve, lstsq
from scipy.sparse import diags

import logging

дискретизации уравнения Пуассона

In [2]:
def makeA(n):
    """
    Функция построениея разрежанной трёхдиагональной матрици,
    где во всех остальных местах, кроме главной диагонали и двух соседних с ней, стоят нули.
    :param n: размер матрицы
    :return: матрица nxn
    """
    return np.diag(2*np.ones(n)) + np.diag(-np.ones(n-1),-1) + np.diag(-np.ones(n-1),1)

In [3]:
def create_NM(n,m):
    """
    #n is the size of the matrix A; m is the number of columns of intial guess
    :param n:
    :param m:
    :return:
    """
    X = np.ones((n,m))
    return X

# Создадим матрицу A и вектор b

In [51]:
# 4096 == 2^12
n = 2**9
m = 1
A = makeA(n)
b = np.ones(n)
block_b = create_NM(n, m)

In [52]:
A.shape

(512, 512)

In [53]:
b.shape

(512,)

In [54]:
block_b.shape

(512, 1)

In [55]:
def _is_matrix_spd(matrix: np.ndarray) -> bool:
    """
    Возвращает True, если входная матрица является симметричной положительно определенной.
    В противном случае возвращается False.
    Для того чтобы матрица была СПО, все собственные значения должны быть положительными.
    :param matrix: матрица nxn
    :return: True/False
    """

    # Проверяем, что матрица квадратная.
    assert np.shape(matrix)[0] == np.shape(matrix)[1], 'Матрицы не квадратная'

    # Если матрица не симметрична, возращаем False
    if np.allclose(matrix, matrix.T) is False:
        return False

    # Получить собственные значения и собственные векторы для симметричной матрицы.
    eigen_values, _ = np.linalg.eigh(matrix)

    # Проверьте знак всех собственных значений.
    # np.all возвращает значение типа np.bool_
    return bool(np.all(eigen_values > 0))

# Блочный Якоби

![block](img/block.png)

In [56]:
def block_jacobi(A: np.array,
                 b: np.array,
                 x: np.array = None,
                 block_size: int = 2,
                 precision: float = 1e-6,
                 max_iter: int = None,
                 verbose: bool = False,
                 return_iter: bool  = False) -> Any:
    """
    разбиваем матрицу A на квадратные блоки, а также вектор b на блоки одинакового размера и
    решает Ax = b. Матрица A разбивается на две матрицы. D содержит диагональные блоки, а L и U
    верхняя и нижняя блочные треугольные матрицы. Параллельное решение блоков.
    Args:
        A (np.array): матрица A линейной системы
        b (np.array): вектор результат Ax
        x (np.array): вектор x
        block_size (int): размер блока
        precision (np.float, опционально): Допуск, если ниже 1e-6(по умолчанию) то прерываем.
        max_iter (int, опционально): максимальное количество итераций алгоритма. По умолчанию 2000.
        verbose (bool, опционально: если true, выводить результаты. По умолчанию False.
        return_iter (bool, опционально): возвращает решение x и количество итераций. По умолчанию False.
    return:
        Union[np.array, int]: решения линейной системы. Если return_iter = True, возвращается число
        итераций.
    """

    # Проверка что размерности подходящие
    assert _is_matrix_spd(A), 'Матрица не является симметричной положительно определенной'
    assert np.shape(A)[0] == np.shape(b)[0]



    if max_iter is None:
        max_iter = len(b)

    iter_counter = 0
    error = np.finfo('double').max

    # размерность блоков
    p = A.shape[0] // block_size
    n = len(A)

    # Расчет диагональных блоков
    A_blocked = A.reshape(p, block_size, p, block_size).swapaxes(1, 2)
    D_blocked = np.diagonal(A_blocked, axis1=0, axis2=1).T

    # рассчитаем LU как диагональный блока A минус матрица A
    LU = block_diag(*D_blocked) - A

    # инициализация вектора x
    if x is None:
        x = np.random.uniform(size=n)

    # запуск блочной итерации якоби
    while error > precision and iter_counter <= max_iter:
        # инициализация x^{k+1}
        x_next = np.zeros(shape=(p, block_size))

        # A_hat - диагональная форма блока (p, размер блока, размер блока)
        A_hat = D_blocked.copy()

        # вычислить b_hat и изменить форму на (p, размер блока)
        b_hat = LU @ x + b
        b_hat = b_hat.reshape(p, block_size)

        # начало параллельного вычисления
        for i in range(block_size-1):
            # доступ ко всем опорным элементам
            pivot = A_hat[:, i, i]

            # исключить все строки, расположенные ниже опорных элементов
            for j in range(i+1, block_size):
                factor = A_hat[:, j, i] / pivot
                A_hat[:, j, :] -= np.multiply(A_hat[:, i, :].T, factor).T
                b_hat[:, j] -= b_hat[:, i] * factor

        # обратная подстановка
        # поэлементное деление
        x_next[:, block_size-1] = b_hat[:, block_size-1] / A_hat[:,block_size-1, block_size-1]
        for k in range(block_size-2, -1, -1):
            # np.einsum('ij,ij->i',...,...) - это np.dot внутри блоков
            x_next[:, k] = (b_hat[:, k] - np.einsum('ij,ij->i', A_hat[:, k, k+1:], x_next[:, k+1:])) / A_hat[:, k, k] # поэлементное деление

        # изменить форму x_next на исходную
        x_next = x_next.reshape(n)
        # расситаем ошибку, норму
        error = norm(x_next - x) / norm(x)
        x = x_next.copy()
        iter_counter += 1

    if error > precision:
        logging.warning(f'Алгоритм завершен т.к достиг максимального количество допустимых итераций. Error = {error}')

    if verbose:
        print(f'# найденное решение {x}')
        print(f'# Нашли за {iter_counter - 1} итераций')
        print(f'# абсолютная относительная погрешность: {error}')

    if return_iter:
        return x, iter_counter-1
    return x

In [57]:
%%time
bj_x = block_jacobi(A=A, b=b, block_size=8, precision=1e-6, max_iter=100000, verbose=True, return_iter=False)

# найденное решение [  254.62280227   508.24560453   760.8684068   1012.49120907
  1263.11401133  1512.7368136   1761.35961587  2008.98241813
  2255.60711174  2501.23342906  2745.85974637  2989.48606368
  3232.112381    3473.73869831  3714.36501562  3953.99133294
  4192.62109569  4430.2542173   4666.88733891  4902.52046052
  5137.15358213  5370.78670374  5603.41982535  5835.05294696
  6065.69130461  6295.33451778  6523.97773095  6751.62094413
  6978.2641573   7203.90737048  7428.55058365  7652.19379682
  7874.84366486  8096.50021839  8317.15677192  8536.81332545
  8755.46987898  8973.12643251  9189.78298605  9405.43953958
  9620.1046236   9833.7777484  10046.4508732  10258.123998
 10468.7971228  10678.47024761 10887.14337241 11094.81649721
 11301.49942246 11507.1922957  11711.88516894 11915.57804219
 12118.27091543 12319.96378867 12520.65666192 12720.34953516
 12919.05415151 13116.76991677 13313.48568202 13509.20144727
 13703.91721252 13897.63297778 14090.34874303 14282.06450828
 14472

In [61]:
bj_Ax = np.dot(A, bj_x)
np.allclose(bj_Ax, b, rtol=1e-2, atol=0.)

False

In [60]:
bj_Ax

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.99810866, 0.9983763 , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.99655456, 0.99664114, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.99476397, 0.99514447,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.99334514, 0.9933145 , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.99146951,
       0.99195922, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.99019956, 0.990052  , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.98825689, 0.9888511 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.98714799, 0.98688493,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.98515692, 0.98584994, 1.        , 1.     

# Блочный Зейделя

In [ ]:
def block_gs( A: np.array,
              b: np.array,
              x: np.array,
              block_size: int = 2,
              precision: float = 1e-6,
              max_iter: int = 2000,
              verbose: bool = False,
              return_iter: bool  = False) -> Any:
    """
    Блочный метод Гаусса-Зейделя
    разбивает матрицу A на квадратные блоки, а также вектор b на блоки одинакового размера и
    решает Ax = b. Матрица A разбивается на две матрицы. D содержит диагональные блоки, а L и U
    это треугольные матрицы верхнего и нижнего блоков.
    Args:
        A (np.array): матрица A линейной системы
        b (np.array): вектор результат Ax
        block_size (int): размер блока
        precision (np.float, опционально): Допуск, если ниже 1e-6(по умолчанию) то прерываем.
        max_iter (int, опционально): максимальное количество итераций алгоритма. По умолчанию 2000.
        verbose (bool, опционально: если true, выводить результаты. По умолчанию False.
        return_iter (bool, опционально): возвращает решение x и количество итераций. По умолчанию False.
    return:
        Union[np.array, int]: решения линейной системы. Если return_iter = True, возвращается число
        итераций.
    """

    # Проверка что размерности подходящие
    assert _is_matrix_spd(A), 'Матрица не является симметричной положительно определенной'
    assert np.shape(A)[0] == np.shape(b)[0]

    iter_counter = 0
    error = np.finfo('double').max
    p = A.shape[0] // block_size
    n = len(A)

    # построиv верхнюю треугольную матрицу
    U = -A.copy()
    for i in range(0,n,block_size):
        U[i:i+block_size, :i+block_size] = 0

    # инициализация вектора x
    if x is None:
        x = np.random.uniform(size=n)

    # начало итераций
    while error > precision and iter_counter <= max_iter:
        # рассчитаем правую часть
        b_hat = U @ x + b
        x_next = np.zeros(n)

        # прямая подстановка
        x_next[0:block_size] = solve(A[:block_size, :block_size], b_hat[:block_size])
        for i in range(1, p):
            block_range = slice(i * block_size, (i + 1) * block_size)
            # получаем диагональный блок
            diag_block = A[block_range, block_range].copy()
            # рассчитаем наклон b
            b_tilted = b_hat[block_range] - A[block_range, :i * block_size] @ x_next[:i * block_size]
            # исключение для значений x. Используется метод solve из scipy.
            x_next[block_range] = solve(diag_block, b_tilted)

        error = norm(x_next - x) / norm(x)
        x = x_next.copy()
        iter_counter += 1


    if error > precision:
        logging.warning(f'Алгоритм завершен т.к достиг максимального количество допустимых итераций. Error = {error}')

    if verbose:
        print(f'# найденное решение\n {x}\n')
        print(f'# Нашли за {iter_counter - 1} итераций')
        print(f'# абсолютная относительная погрешность\n {error}')

    if return_iter:
        return x, iter_counter-1
    return x

In [ ]:
%%time
bgs_x = block_gs(A=A, b=b, block_size=8, precision=1e-10, max_iter=10000, verbose=True, return_iter=False)

In [ ]:
bgs_Ax = np.dot(A, bgs_x)

np.allclose(bgs_Ax, b, rtol=1e-10, atol=0.)

# Метод сопряжённых градиентов

![](img/CG.png)

In [9]:
def CG(A: np.array,
       b: np.array,
       x: np.array = None,
       tol: float = 1.e-10,) -> np.array:
    """
    Метод сопряжённых градиентов (для решения СЛАУ)
    :param A: матрицв nxn
    :param b: вектор RHS n
    :param x: искомый вектор n
    :param tol: погрешность
    :return: найденный вектор x
    """

    # Проверка что размерности подходящие
    assert _is_matrix_spd(A), 'Матрица не является симметричной положительно определенной'
    assert np.shape(A)[0] == np.shape(b)[0]

    if x is None:
        x = np.zeros(A.shape[-1])

    # невязка
    r = b - np.dot(A, x)

    # направление
    p = r

    # итераций
    k = 0

    r_old  = np.dot(r.T,r)

    while np.linalg.norm(r) > tol:

        A_p = A @ p
        alpha = r_old/np.dot(p.T, A_p)

        x = x + alpha*p

        r = r - alpha*A_p

        beta = np.dot(r.T, r)/r_old
        p = r + beta*p

        #update the denominator to be the numerator's value for the next fraction
        r_old = np.dot(r.T,r)

        k += 1

    print(f'Заняло {k} итераций')
    return x

In [10]:
%%time
CG_x = CG(A, b)

Заняло 2048 итераций
CPU times: user 1min 14s, sys: 1.45 s, total: 1min 15s
Wall time: 1min 4s


In [11]:
CG_Ax = np.dot(A, CG_x)
np.allclose(CG_Ax, b, rtol=1e-8, atol=0.)

True

# Блочный метод сопряжённых градиентов

![BCG](img/BCG.png)

In [12]:
def BCG(A: np.array,
        B: np.array = None,
        X: np.array = None,
        tol: float = 1e-8,
        max_iter: int = None,
        block_size: int = 2) -> np.array:
    """
    # Блочный метод сопряжённых градиентов
    :param A: матрица nxn
    :param B: вектор nx1
    :param X: искомый вектор nx1
    :param tol: точность
    :param max_iter: количествой итераций
    :param block_size: размер блока
    :return: вектор x
    """
    # Проверка что размерности подходящие
    assert _is_matrix_spd(A), 'Матрица не является симметричной положительно определенной'
    assert np.shape(A)[0] == np.shape(b)[0]

    if X is None:
        X = create_NM(n, block_size)

    if B is None:
        B = create_NM(n, block_size)

    if max_iter is None:
        max_iter = len(A[0])

    # Невязка
    R = B - A@X

    # Направление
    P = R

    # итераций
    k = 0

    # знаменатель (для облегчения последующих расчетов)
    R_old = R.T @ R

    #  норма Фробениуса
    while k < max_iter and np.max(np.linalg.norm(R, axis=0)) > tol:

        A_P = A @ P
        P_A_P_inv = np.linalg.pinv(P.T @ A_P)

        Lam = P_A_P_inv @ R_old
        PLam = P @ Lam

        X = X + PLam
        R = R - A_P @ Lam

        R_T_R = R.T @ R
        Phi= np.linalg.pinv(R_old) @ R_T_R

        P = R + P@Phi
        # обновить знаменатель, чтобы он стал значением числителя для следующей дроби
        R_old = R.T@R

        k+=1

    print(f'Заняло {k} итераций')
    return X


In [13]:
%%time
BCG_x = BCG(A, block_size=1)

Заняло 2048 итераций
CPU times: user 1min 12s, sys: 1.19 s, total: 1min 13s
Wall time: 49.5 s


In [14]:
BCG_Ax = np.dot(A, BCG_x)

In [15]:
np.allclose(BCG_Ax[:,0], block_b, rtol=1e-7, atol=0.)

True

# Метод сопряжённых градиентов c предобусловленной системы
- [Неполная факторизация Холески](https://en.wikipedia.org/wiki/Incomplete_Cholesky_factorization)
- [Jacobi](https://www.cse.psu.edu/~b58/cse456/lecture20.pdf)
- [SSOR](https://math-linux.com/mathematics/linear-systems/article/preconditioned-conjugate-gradient-method)

![PCG](img/PCG.png)

In [16]:
def incomplete_cholesky_factorization(A:np.array) -> Any:
    """
    Вычисляем неполную факторизацию Холески для матрицы A
    :param A: матрица nxn
    :return: матрица
    """

    # Проверка что размерности подходящие
    assert _is_matrix_spd(A), 'Матрица не является симметричной положительно определенной'

    M = np.copy(A)
    n = M.shape[1]

    for k in range(n):
        M[k, k] = np.sqrt(M[k, k])
        for i in range(k+1, n):
            if M[i, k] != 0:
                M[i, k] = M[i, k] / M[k, k]
        for j in range(k+1, n):
            for i in range(j, n):
                if M[i, j] != 0:
                    M[i, j] = M[i, j] - M[i, k] * M[j, k]
    for i in range(n):
        for j in range(i+1, n):
            M[i, j] = 0

    return M

In [17]:
def splitMat(A: np.array) -> Any:
    n,m = A.shape
    if (n == m):
        diagval = np.diag(A)
        D = diags(diagval,0).toarray()
        L = (-1)*np.tril(A,-1)
        U = (-1)*np.triu(A,1)
    else:
        print("A needs to be a square matrix")
    return L, D, U

def P_SSOR(A: np.array,
           w: float = 2/3) -> np.array:
    """
    SSOR Preconditioner(Symmetric Successive Over Relaxation)
    :param A: Матрица
    :return: Матрица
    """
    L,D,U = splitMat(A)
    P = 2/(2-w) * (1/w*D + L) * np.linalg.inv(D) * (1/w*D + L).T
    return P

In [18]:
def PCG(A: np.array,
        b: np.array,
        x: np.array = None,
        tol: float = 1e-10,
        precond: str= 'jacobi') -> np.array:

    """
    Метод сопряжённых градиентов c предобусловленной системы
    Неполная факторизация Холецкого или Якоби
    :param A: матрица nxn
    :param b: вектор nx1
    :param x: вектор nx1
    :param tol: погрешность
    :param precond: тип предобусловленной системы, по умолчанию неполная факторизация Холецкого
    :return: вектор x nx1
    """
    if x is None:
        x = np.zeros(A.shape[-1])

    # невязка
    r = b - A.dot(x)

    # M^-1
    precondition = None
    if precond  in ['cholesky', 'chol']:
        # вычисление неполной факторизации Холески для A в качестве предобусловливателя
        precondition = np.linalg.inv(incomplete_cholesky_factorization(A))

    if precond in ['jacobi', 'jac']:
        # предобуславливание Якоби
        precondition = np.linalg.inv(np.diag(A.diagonal()))

    if precond in ['SSOR', 'ssor']:
        precondition = np.linalg.inv(P_SSOR(A))

    # вектор направлений
    z = np.dot(precondition, r)
    p = z

    # погрешность
    error = norm(r)

    # счетчик итераций
    k = 0

    # условия сходимости
    while error > tol:

        Ap = np.dot(A, p)
        alpha = np.dot(r.T, z) / np.dot(p.T, Ap)

        phi = np.dot(z.T,  r)
        old_res = z

        x = x + alpha * p
        r = r - alpha * Ap

        z = np.dot(precondition, r)
        beta = np.dot(z.T, (r - old_res)) / phi # формула Polak–Ribière

        p = z + beta * p
        error = r.T.dot(r)

        k += 1

    print(f'Заняло {k} итераций')
    return x

In [137]:
%%time
PCG_x = PCG(A, b, precond='cholesky')

KeyboardInterrupt: 

In [138]:
PCG_Ax = np.dot(A, PCG_x)
np.allclose(PCG_Ax, b, rtol=1e-10, atol=0.)

NameError: name 'PCG_x' is not defined

In [19]:
%%time
PCG_j_x = PCG(A, b, precond='jacobi')

Заняло 2048 итераций
CPU times: user 1min 33s, sys: 1.46 s, total: 1min 35s
Wall time: 1min 8s


In [20]:
PCG_j_Ax = np.dot(A, PCG_j_x)
np.allclose(PCG_j_Ax, b, rtol=1e-10, atol=0.)

True

In [21]:
%%time
PCG_ssor_x = PCG(A, b, precond='ssor')

Заняло 2048 итераций
CPU times: user 1min 38s, sys: 1.73 s, total: 1min 40s
Wall time: 58.1 s


In [22]:
PCG_ssor_Ax = np.dot(A, PCG_ssor_x)
np.allclose(PCG_ssor_Ax, b, rtol=1e-6, atol=0.)

True

# Блочный метод сопряжённых градиентов c предобусловленной системы

![PBCG](img/PBCG.png)

In [23]:
def PBCG(A: np.array,
         B: np.array = None,
         X: np.array = None,
         tol: float = 1e-6,
         max_iter: int = None,
         precond: str = None,
         block_size: int = 1) -> np.array:
    """
    Блочный метод сопряжённых градиентов c предобусловленной системы
    :param A: матрица nxn
    :param B: вектор nxblock_size
    :param X: искомый вектор nxblock_size
    :param tol: погрешность
    :param max_iter: количество итераций
    :param precond: система предобусловленния
    :param block_size: размер блока
    :return: x
    """

    if X is None:
        X = create_NM(n, block_size)
    if B is None:
        B = create_NM(n, block_size)
    if max_iter is None:
        max_iter = len(A[0])

    # M^-1
    if precond  in ['cholesky', 'chol']:
        # вычисление неполной факторизации Холески для A в качестве предобусловливателя
        Minv = np.linalg.inv(incomplete_cholesky_factorization(A))

    if precond in ['jacobi', 'jac']:
        # предобуславливание Якоби
        Minv = np.linalg.inv(np.diag(A.diagonal()))

    if precond in ['SSOR', 'ssor']:
        Minv = np.linalg.inv(P_SSOR(A))

    # невязка
    R = B-A.dot(X)

    # направление
    Z = Minv @ R
    P = Z

    # количество итераций
    k = 0

    # для будущих вычислений
    R_old  = R.T@Z

    while k < max_iter and np.max(np.linalg.norm(R, axis=0)) > tol:

        A_P = A@P
        P_A_P_inv = np.linalg.pinv(P.T@A_P)
        Lam = P_A_P_inv@ R_old
        PLam = P @ Lam

        X = X + PLam
        R = R - A @ PLam

        Z = Minv@R
        R_T_Z = R.T@Z
        Phi = np.linalg.pinv(R_old) @ R_T_Z

        P = Z + P@Phi
        #обновить знаменатель, чтобы он стал значением числителя для следующей дроби
        R_old = R.T@Z

        k+=1

    print(f'Заняло {k} итераций')
    return X


In [ ]:
%%time
PBCG_x = PBCG(A, precond='cholesky', block_size=2)

In [ ]:
PBCG_Ax = np.dot(A, PBCG_x)
np.allclose(PBCG_Ax[:, 0], block_b, rtol=1e-10, atol=0.)

In [24]:
%%time
PBCG_j_x = PBCG(A, precond='jacobi', block_size=2)

Заняло 2048 итераций
CPU times: user 3min 58s, sys: 2.33 s, total: 4min
Wall time: 2min 6s


In [27]:
PBCG_j_Ax = np.dot(A, PBCG_j_x)
np.allclose(PBCG_j_Ax, block_b, rtol=1e-6, atol=0.)

True

In [28]:
%%time
PBCG_ssor_x = PBCG(A, precond='ssor', block_size=2)

Заняло 2048 итераций
CPU times: user 4min 20s, sys: 3.95 s, total: 4min 24s
Wall time: 2min 45s


In [29]:
PBCG_ssor_Ax = np.dot(A, PBCG_ssor_x)
np.allclose(PBCG_ssor_Ax[:, 0], block_b, rtol=1e-6, atol=0.)

True

# Обобщенный метод минимальный невязок

In [30]:
def Step_Arnoldi(A: np.array,
                 Q: np.array,
                 H: np.array) -> Any:
    """
    Итерация Арнольди
    :param A:
    :param Q:
    :param H:
    :return:
    """
    n, k = np.shape(Q)
    h = np.zeros((k+1, 1))
    q = np.zeros((n, 1))
    p = A.dot(Q[:,-1].reshape((n,1)))
    h[0:k,0] = Q.T.dot(p).squeeze()
    p -= Q.dot(h[0:k,0].reshape(k,1))
    h[k,0] = norm(p)

    if H is None:
        H = h
    else:
        m = np.zeros((1,k-1))
        H = np.concatenate((H,m),axis = 0)
        H = np.concatenate((H,h),axis = 1)
    p = p/h[k,0]
    q[0:n,0] = p.squeeze()
    Q = np.concatenate((Q,q),axis = 1)

    return Q,H


def GMRES(A,
          b,
          x0 = None,
          tol = 1e-10,
          itermax = None):
    """
    Обобщенный метод минимального невязок
    :param A:
    :param b:
    :param x0:
    :param tol:
    :param itermax:
    :return:
    """
    if x0 is None:
        x0 = np.zeros((len(b),1))

    if itermax is None:
        itermax = len(b)

    b = b.reshape((len(b),1))
    r = b - A.dot(x0)
    beta = norm(r)
    rho = beta
    Q = r.reshape((len(b),1))/float(beta)
    H = None
    niter = 1

    while (niter<itermax) and (rho>tol):
        Q, H = Step_Arnoldi(A,Q,H)
        e1 = np.zeros(niter+1)
        e1[0] = 1
        y = lstsq(a=H, b=e1*beta)[0]
        y = y.reshape(niter,1)
        x = x0 + Q[:,:niter].dot(y)
        r = b - A.dot(x)
        rho = norm(r)
        niter += 1
    return x

In [31]:
%%time
x1= GMRES(A, b)

KeyboardInterrupt: 

In [ ]:
Ax1 = np.dot(A, x1)
np.allclose(Ax1, b, rtol=1e-06, atol=0.)

# Многосеточный метод

In [ ]:
def Jacobi(x, f, omega, N):
    """
    # запустить взвешенный метод Якоби для одномерного Пуассона с параметром omega
    :param x:
    :param f:
    :param omega:
    :param N:
    :return:
    """

    dx2        = 1/(N**2)
    omega_min1 = 1.0-omega
    omega_div2 = 0.5*omega
    x_new      = x[:]

    for i in range(1,N):
        x_new[i] = omega_min1*x[i] + omega_div2*(x[i-1] + x[i+1] + dx2*f[i])

    return x_new



def residual(x, f, N):
    """
    вычислить невязку f-Ax для приближенного решения x уравнения Ay=b
    :param x:
    :param f:
    :param N:
    :return:
    """
    res      = np.zeros(N+1)
    res[1:N] = [f[i] - N**2 * (2*x[i] - x[i-1] - x[i+1])
                for i in range(1, N)]

    return res




def coarsen(res_fine, N):
    """
    грубая обработка вектора и сетки, где вектор определен через ограничение
    :param res_fine:
    :param N:
    :return:
    """

    M          = int((N+1) / 2)
    res_coarse = np.zeros(M+1)
    res_coarse = [res_fine[2*i] for i in range(0,M+1)]

    return res_coarse, M


def refine(res_coarse, N):
    """
    уточняем вектор и сетку, где вектор определяется с помощью интерполяции
    :param res_coarse:
    :param N:
    :return:
    """

    res_fine = np.zeros(2*N+1)
    res_fine[0:2*N:2] = [res_coarse[i] for i in range(0, N)]
    res_fine[1:2*N:2] = [0.5*(res_coarse[i] + res_coarse[i+1]) for i in range(0, N)]
    res_fine[2*N]     = res_coarse[N]

    return res_fine, 2*N




def V_cycle(x, f, N, sweep_start, sweep_end, omega):
    """
    многосеточный V-цикл
    :param x:
    :param f:
    :param N:
    :param sweep_start:
    :param sweep_end:
    :param omega:
    :return:
    """

    # запуск генератора развертки для снижения
    for i in range(sweep_start):

        y = Jacobi(x,f, omega, N)
        x = y[:]

    # вычислить остаток, затем огрубить его
    res             = residual(y, f, N)
    [res_coarse, N] = coarsen(res, N)


    # если проблема достаточно велика, рекурсивно примените V-цикл к остаточному уравнению,
    # затем уточнить
    if N>4:

        x0          = np.zeros(N+1)
        x_coarse    = V_cycle(x0, res_coarse, N, sweep_start,
                              sweep_end, omega)
        [x_fine, N] = refine(x_coarse, N)

    # если N достаточно мало, решаем напрямую
    else:

        # установим матрицу Пуассона A, затем выполним прямое решение остаточного уравнения
        band         = [-1*np.ones(N-2), 2*np.ones(N-1), -1*np.ones(N-2)]
        offset       = [-1, 0, 1]
        A            = (N**2) * diags(band,offset).toarray()
        exact        = np.zeros(N+1)
        exact[1:N]   = np.linalg.solve(A, res_coarse[1:N])
        [x_fine,  N] = refine(exact, N)

        # правильная аппроксимация с оценкой ошибок

    y       = np.add(x_fine, y)
    x_final = np.zeros(N+1)


    # запуск relax
    for i in range(sweep_end):

        x_final = Jacobi(y, f, omega,N)
        y       = x_final[:]

    return x_final



In [ ]:
# установите параметры многосеточной сетки и границы
omega       = 2/3
sweep_start = 3
sweep_end   = 3
cycles      = 8
g           = np.random.rand(2)
N = 2**10

In [ ]:
x    = np.zeros(N+1)
f    = np.random.rand(N+1)
f[0] = 0
f[N] = 0
x[0] = g[0] # изменим левую границу
x[N] = g[1] # изменим правую границу

for i in range(cycles): # запуск цикл V-цикла

    y  = V_cycle(x, f, N, sweep_start, sweep_end, omega)
    x  = y[:]


In [ ]:
f

In [ ]:
# точное решение
band       = [-1*np.ones(N-2), 2*np.ones(N-1), -1*np.ones(N-2)]
offset     = [-1, 0, 1]
A          = diags(band, offset).toarray()
ff         = (N**-2) * f[1:N]
x_dir      = np.zeros(N+1)
x_dir[0]   = g[0]
x_dir[N]   = g[1]
gg         = np.zeros(N-1)
gg[0]      = g[0]
gg[N-2]    = g[1]
RHS        = np.add(ff, gg)
x_dir[1:N] = np.linalg.solve(A, RHS)

In [ ]:
x_dir

In [ ]:
x

# Методы подпространств Крылова
- Алгоритм Ланцоша
- Алгоритм Арнольди

## Арнольди

In [ ]:
def Step_Arnoldi(A,
                 Q,
                 H):
    """

    :param A:
    :param Q:
    :param H:
    :return:
    """
    n,k = np.shape(Q)
    h = np.zeros((k+1,1))
    q = np.zeros((n,1))
    p = A.dot(Q[:,-1])
    h[0:k,0] = Q.T.dot(p)
    p -= Q.dot(h[0:k,0])
    h[k,0] = norm(p)
    if H is None:
        H = h
    else:
        m = np.zeros((1,k-1))
        H = np.concatenate((H,m),axis = 0)
        H = np.concatenate((H,h),axis = 1)
    p = p/h[k,0]
    q[0:n,0] = p
    Q = np.concatenate((Q,q),axis = 1)
    return Q,H

In [ ]:
def QR_HessenH(M):
    """

    :param M:
    :return:
    """
    n = np.shape(M)[0]
    U = np.eye(n)
    R = np.copy(M)
    for k in range(n-1):
        x = R[k:k+2,k]
        l = norm(x)
        c = x[0]/l
        s = x[1]/l
        for P in [U,R]:
            P_tmp_k = np.copy(P[k,:])
            P_tmp_kp1 = np.copy(P[k+1,:])
            P[k,:] = c*P_tmp_k + s*P_tmp_kp1
            P[k+1,:] = c*P_tmp_kp1 - s*P_tmp_k
    Q = U.T
    return Q,R

def Eig_QR_Defla(A, tol = 1e-7, itermax = None):
    """

    :param A:
    :param tol:
    :param itermax:
    :return:
    """
    n,m = np.shape(A)
    if n < 2:
        return A,0
    if itermax is None:
        itermax = 1000*n
    H = np.copy(A)
    mu = H[n-1,n-1]
    k = 0
    I = np.eye(n)
    Sd = np.zeros((n-1,1))
    L = []
    while (k<itermax):
        k += 1
        Q,R = QR_HessenH(H - mu*I)
        H = R.dot(Q) + mu*I
        H = np.triu(H,-1)
        mu = H[n-1,n-1]
        for l in range(n-1):
            Sd[l] = np.copy(H[l+1,l])
        i = np.argmin(abs(Sd))
        if abs(Sd[i])<tol:
            L1,ks1 = Eig_QR_Defla(H[0:i+1,0:i+1], tol, itermax)
            L2,ks2 = Eig_QR_Defla(H[i+1:n,i+1:n], tol, itermax)
            L = np.append(L,L2)
            L = np.append(L,L1)
            k = k + ks1 + ks2
            return(L,k)
    return np.diag(H),k


In [ ]:
def Arnoldi(A,
            b = None,
            p = 1,
            tol = 1e-8):
    """

    :param A:
    :param b:
    :param p:
    :param tol:
    :return:
    """
    n,m = np.shape(A)
    if b is None:
        b = np.random.random((n,1))
    k = 0
    b = np.reshape(b,(n,1))
    Q = np.zeros((n,1))
    Q[:,0] = b[:,0]/norm(b)
    H = None
    L = []
    lt = 0
    for k in range(n):
        k += 1
        Q,H = Step_Arnoldi(A,Q,H)
        l,niter = Eig_QR_Defla(H[0:k,0:k])
        d = abs(np.subtract.outer(l,lt))
        if np.size(d[d<tol]) >= p:
            L += [l]
            return L,k
        lt = l
        L += [l]
    return L,k

In [ ]:
%%time
L, k =  Arnoldi(A, b)

## Алгоритм Ланцоша

In [ ]:
def Step_Lanczos(A,
                 Q,
                 T):
    """

    :param A:
    :param Q:
    :param T:
    :return:
    """
    if T is None:
        n,k = np.shape(Q)
        v = A.dot(Q[:,0])
        a = Q[:,0].dot(v)
        v -= a*Q[:,0]
        v = np.reshape(v,(n,1))
        b = norm(v)
        Q = np.concatenate((Q,v/b),axis = 1)
        T = np.zeros((2,1))
        T[0,0] = a
        T[1,0] = b
    else:
        n,k = np.shape(Q)
        v = A.dot(Q[:,k-1]) - T[k-1,k-2]*Q[:,k-2]
        a = Q[:,k-1].dot(v)
        v -= a*Q[:,k-1]
        v = np.reshape(v,(n,1))
        b = norm(v)
        Q = np.concatenate((Q,v/b),axis = 1)
        m = np.zeros((1,k-1))
        T = np.concatenate((T,m),axis = 0)
        l = np.zeros((k+1,1))
        l[k-2,0] = T[k-1,k-2]
        l[k-1,0] = a
        l[k,0] = b
        T = np.concatenate((T,l),axis = 1)
    return Q,T

In [ ]:
def Lanczos(A,
            b = None,
            p = 1,
            tol = 5e-1):
    """

    :param A:
    :param b:
    :param p:
    :param tol:
    :return:
    """
    n,m = np.shape(A)
    if b is None:
        b = np.random.random((n,1))
    k = 0
    b = np.reshape(b,(n,1))
    Q = np.zeros((n,1))
    Q[:,0] = b[:,0]/norm(b)
    H = None
    Lt = 0
    for k in range(n):
        k += 1
        Q,H = Step_Lanczos(A,Q,H)
        L,niter = Eig_QR_Defla(H[0:k,0:k])
        d = abs(np.subtract.outer(L,Lt))
        if np.size(d[d<tol]) >= p:
            return L,k
        Lt = L
    return L,k

In [ ]:
L, k = Lanczos(A, b)

In [ ]:
np.array([[1,0],
          [0,2]]) @ np.array([1,1])